In [1]:
import logging
from typing import Union, Optional, NoReturn
from web3 import Web3
from web3.middleware import geth_poa_middleware
import aiohttp
import asyncio
import pandas as pd
import pprint
import json
import numpy as np

logger = logging.getLogger(__name__)
chain='bsc'
ver='v3'
rpc = 'https://bsc-dataseed.binance.org/'
web3 = Web3(Web3.HTTPProvider(rpc))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
tokens = {}
base_token = 'BNB'
base_token_dict = {}

async def fetch(sess, url, params={}):

    async with sess.get(url, params=params) as resp:
        obj = await resp.text()
        obj = json.loads(obj)
        # print(obj)
        logger.debug(f'Fetch ==> {obj}')
        # print(f'Fetch ==> {obj}')

        return obj

def get_response_result(obj,
                        raise_error: bool = True,
                        result_prop: str = 'data') -> Optional[dict]:
    """Receives the response body from the server, then returns an object
    located in result_prop, or throws an exception if from the server
    received error information
    """

    if obj['code'] == 200:
        if result_prop == 'full':
            return obj
        else:
            return obj[result_prop]

    else:
        keep_alive = False
        print('Error found!')
        print(f'Error: code: {obj["code"]}')
        print(f'Error: msg: {obj}')
        logger.debug('Error found!')
        logger.debug(f'Error: code: {obj["code"]}')
        logger.debug(f'Error: msg: {obj[result_prop]}')

        # if raise_error:
        #     raise CBotResponseError(obj[result_prop],obj['code'])

        return None

async def getTokens(sess) -> Optional[dict]:
    global tokens, base_token, base_token_dict

    logger.info('get_tokens')
    print('get_tokens')
    url = 'open-api.openocean.finance'
    url = f'https://{url}/{ver}/{chain}/tokenList'
    tokens = get_response_result(await fetch(sess, url))
    tokens = pd.DataFrame(tokens)
    tokens['amount'] = np.nan
    tokens['ret_amount'] = np.nan

    tokens = tokens.loc[:50]
    # tokens.set_index('id', drop=False, inplace=True)

    base_token_dict = tokens[tokens['symbol'] == base_token].to_dict('records')[0]
    tokens = tokens.to_dict('index')

async def quote(sess, base, qt, amt):
    print(f'quote: {base} {qt} {amt}')
    url = 'open-api.openocean.finance'
    url = f'https://{url}/{ver}/{chain}/quote'

    parms = {
        'chain'          : 'bsc',
        'inTokenAddress' : base,
        'outTokenAddress': qt,
        'amount'         : amt,
        'gasPrice'       : 5,
        'slippage'       : 1
    }

    return get_response_result(await fetch(sess, url, params=parms))

async def init_quote(sess, key):
    global tokens, base_token, base_token_dict
    print(f'init_quote {key}')

    amt = 4
    try:
        res = await quote(sess, base_token_dict['address'], tokens[key]['address'], amt)
        # convert outAmount to ether
        # print(f"wei: {res['outAmount']}")
        # amtFromWei = web3.fromWei(res['outAmount'], "ether")
        # tokens[key]['amount'] = float(amtFromWei)
        tokens[key]['amount'] = res['outAmount']
        print(f"ether: {res['outAmount']}")

    except Exception as E:
        print(f'key: {key} Error1: {E}')

async def ret_quote(sess, key):
    global tokens, base_token, base_token_dict
    print(f'ret_quote {key}')

    try:
        res = await quote(sess, tokens[key]['address'], base_token_dict['address'], tokens[key]['amount'])
        amtFromWei = web3.fromWei(res['outAmount'], "ether")
        tokens[key]['ret_amount'] = float(amtFromWei)

    except Exception as E:
        print(f'key: {key} Error2: {E}')


In [30]:
async def main():
    async with aiohttp.ClientSession() as session:
        res = await getTokens(session)
        res = pd.DataFrame(res)
        res.set_index('id', drop=False, inplace=True)
        res = res.to_dict('index')
        print(res)
        return res

async def test():

        async with aiohttp.ClientSession() as session:
            try:
                await getTokens(session)
                await asyncio.sleep(10)
                tasks = []

                for key in tokens:
                    tasks.append(asyncio.create_task(init_quote(session, key)))

                await asyncio.gather(*tasks)

                await asyncio.sleep(3)

                # tasks = []
                # for key in tokens:
                #     tasks.append(asyncio.create_task(ret_quote(session, key)))

                # await asyncio.gather(*tasks)

                # await asyncio.sleep(3)


            except Exception as E:
                print('Error')
                print(E)


data = await test()

get_tokens
init_quote 0
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x37a56cdcD83Dce2868f721De58cB3830C44C6303 4
init_quote 1
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x52F24a5e03aee338Da5fd9Df68D2b6FAe1178827 4
init_quote 2
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82 4
init_quote 3
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d 4
init_quote 4
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x1af3f329e8be154074d8769d1ffa4ee058b1dbc3 4
init_quote 5
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c 4
init_quote 6
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x086ddd008e20dd74c4fb216170349853f8ca8289 4
init_quote 7
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x98936bde1cf1bff1e7a8012cee5e2583851f2067 4
init_quote 8
quote: 0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE 0x44ec807ce2f4a6f2737a92e985f318d035883e47 4
init_quote 9
quot

In [31]:
df = pd.DataFrame(tokens.values())

In [32]:
df

,id,code,name,address,decimals,symbol,icon,chain,createtime,hot,sort,chainId,customSymbol,customAddress,usd,amount,ret_amount
0,2128,zebec-protocol,Zebec Protocol,0x37a56cdcD83Dce2868f721De58cB3830C44C6303,9,ZBC,https://cloudstorage.openocean.finance/images/...,bsc,2022-12-19T14:06:14.000Z,None,2022-12-19T14:06:14.000Z,NaN,None,None,0.01034228,97613559902353,NaN
1,2125,ankr-staked-bnb,Ankr Staked BNB,0x52F24a5e03aee338Da5fd9Df68D2b6FAe1178827,18,ankrBNB,https://cloudstorage.openocean.finance/images/...,bsc,2022-12-14T09:00:46.000Z,None,2022-12-14T09:00:46.000Z,NaN,None,None,264.71,3827520591495527329,NaN
2,499,pancakeswap-token,PancakeSwap Token,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,18,CAKE,https://ethapi.openocean.finance/logos/bsc/0x0...,bsc,2021-10-19T10:40:52.000Z,01,2022-12-08T04:24:09.000Z,56.0,None,None,3.25,314079296113976640678,NaN
3,629,usd-coin,Binance-Peg USD Coin,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,18,USDC,https://ethapi.openocean.finance/logos/bsc/0x8...,bsc,2021-10-19T10:40:52.000Z,01,2022-12-08T04:24:00.000Z,56.0,None,None,1.001,1024378156599877658063,NaN
4,510,dai,Binance-Peg Dai Token,0x1af3f329e8be154074d8769d1ffa4ee058b1dbc3,18,DAI,https://ethapi.openocean.finance/logos/bsc/0x1...,bsc,2021-10-19T10:40:52.000Z,01,2022-12-08T04:23:50.000Z,56.0,None,None,0.998853,1024741513079678070940,NaN
5,634,wbnb,Wrapped BNB,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,18,WBNB,https://cloudstorage.openocean.finance/images/...,bsc,2021-10-19T10:40:52.000Z,01,2022-12-08T04:23:24.000Z,56.0,None,None,256.5,NaN,NaN
6,2118,mxmboxceus-token,MxmBoxcEus Token,0x086ddd008e20dd74c4fb216170349853f8ca8289,18,MBE,https://cloudstorage.openocean.finance/images/...,bsc,2022-12-08T04:11:15.000Z,01,2022-12-08T04:11:15.000Z,NaN,None,None,0.04102568,24780902988971507059542,NaN
7,2105,annex,Annex Finance,0x98936bde1cf1bff1e7a8012cee5e2583851f2067,18,ANN,https://cloudstorage.openocean.finance/images/...,bsc,2022-11-23T06:33:16.000Z,None,2022-11-23T06:33:16.000Z,56.0,None,None,0.00350338,270597146304175593746993,NaN
8,2104,hashflow,Hashflow,0x44ec807ce2f4a6f2737a92e985f318d035883e47,18,HFT,https://cloudstorage.openocean.finance/images/...,bsc,2022-11-22T11:45:17.000Z,None,2022-11-22T11:45:17.000Z,56.0,None,None,0.292686,3479686773060842932439,NaN
9,2101,multichain,Multichain,0x9fb9a33956351cf4fa040f65a13b835a3c8764e3,18,MULTI,https://cloudstorage.openocean.finance/images/...,bsc,2022-11-22T11:27:12.000Z,None,2022-11-22T11:27:12.000Z,56.0,None,None,4.47,NaN,NaN


In [19]:
res['outAmount']

'97617928323401'

In [98]:
async def chk1():
    print('start chk1')
    await asyncio.sleep(5)
    print('end chk1')

async def chk2():
    print('start chk2')
    await asyncio.sleep(2)
    print('end chk2')

async def chk3():
    # await asyncio.gather(chk1(), chk2())
    await chk1()
    await chk2()

await chk3()

start chk1
end chk1
start chk2
end chk2


In [2]:
price = 17413.0

lprice = price - price % 250
hprice = lprice + 250

print(lprice, hprice)

17250.0 17500.0


In [4]:
async def cal1():
    await asyncio.sleep(1)
    return 1

async def cal2():
    await asyncio.sleep(2)
    return 2

async def cal3():
    await asyncio.sleep(3)
    return 3

x = await asyncio.gather(
    cal3(),
    cal2(),
    cal1()
)
x


[3, 2, 1]

In [4]:
from datetime import date, datetime, timedelta, timezone
import numpy as np

today = date.today() #datetime(2023, 2, 1)
friday = today + timedelta( (3-today.weekday()) % 7+1 )
print(f"The date is : {friday.strftime(f'{friday.day}%b%y').upper()}")
print(f"The weekday : {today.weekday() == 4}")

The date is : 10FEB23
The weekday : False


In [2]:
friday.timestamp()

AttributeError: 'datetime.date' object has no attribute 'timestamp'

In [9]:
x = np.arange(0.0005, 0.015, 0.0005)
eval = lambda x: (x * 0.6) - (0.015 * 0.4)
loss = 0.015 * 0.2
x * 0.8 - loss

array([-0.0026, -0.0022, -0.0018, -0.0014, -0.001 , -0.0006, -0.0002,
        0.0002,  0.0006,  0.001 ,  0.0014,  0.0018,  0.0022,  0.0026,
        0.003 ,  0.0034,  0.0038,  0.0042,  0.0046,  0.005 ,  0.0054,
        0.0058,  0.0062,  0.0066,  0.007 ,  0.0074,  0.0078,  0.0082,
        0.0086])

In [13]:
import random
import numpy as np

# range(1948, 2022, 1)
# np.linspace(1948, 2022, )
len(np.arange(1948, 2022, 1))

74

In [ ]:
for i in range(10)